In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

# Major concepts of this program were inspired by following Samson Zhang's NN from Scratch Youtube video
# I created my own datasets, all images were drawn by hand
# I found the script below to turn a set of images into a np array, and tweaked it a little

In [6]:
DATADIR = "/kaggle/input/trainingdata/TrainingData"
CATEGORIES = ["Happy", "Sad"]

data = []
IMG_SIZE = 50

## will create a npy array that stores our training data
def create_training_data():
    
    class_num = 0
    
    for category in CATEGORIES:
        
        ## this for loop iterates through each image in my uploaded Kaggle dataset and appends it to a 2D npy array
        
        path = os.path.join(DATADIR, category)
        for img in os.listdir(path):
            img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
            new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
            flattened_pixels = new_array.flatten().tolist()  # Flatten and convert to a Python list

            data.append([class_num] + flattened_pixels)
        class_num += 1
            
create_training_data()

In [ ]:
data = np.array(data)
m, n = data.shape

## it is important to shuffle data, because each class was in a different directory
## create_training_data() was done in order so the data array would be [0,0,0, ... ,1,1,1]
np.random.shuffle(data)

## this set of array manipulation separates the labels from actual image pixel data
## we transpose the data_dev and data_train arrays to allow for matrix multiplication
data_dev = data[0:20].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[20:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape


## this simple neural network will only have one hidden layer and one output layer (so 3 layers total)
## all weights and biases are set to a random number between -0.5 and 0.5
def init_params():
    W1 = np.random.rand(10, 2500) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(2, 10) - 0.5
    b2 = np.random.rand(2, 1) - 0.5
    
    return W1, b1, W2, b2


## ReLU and softmax are some good functions to emphasize the change between each iteration
## Samson also used these as examples in his video
def ReLU(Z):
    return np.maximum(0, Z)

def softmax(Z):
    return np.exp(Z) / sum(np.exp(Z))

## Applies weights and biases to an epoch of images (all training images are in the data array, passed as X here)
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    
    return Z1, A1, Z2, A2

## 
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def deriv_ReLU(Z):
    return Z > 0

def back_prop(Z1, A1, Z2, A2, W2, X, Y):  
    m = Y.size
    
    one_hot_Y = one_hot(Y)
    
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2, axis=1).reshape(-1, 1)
    dZ1 = W2.T.dot(dZ2) * deriv_ReLU(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1, axis=1).reshape(-1, 1)
    
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    
    return W1, b1, W2, b2



def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, iterations, alpha):
    W1, b1, W2, b2 = init_params()
    m = len(Y)


    for i in range(iterations):

        if(i%1==0):
            permutation = np.random.permutation(m)
            X_shuffled = X.T[permutation]
            Y_shuffled = Y[permutation]
            alpha *= 0.9999

        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X_shuffled.T)
        dW1, db1, dW2, db2 = back_prop(Z1, A1, Z2, A2, W2, X_shuffled.T, Y_shuffled)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)

        if i % 1000 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            accuracy = get_accuracy(predictions, Y_shuffled) #accuracy = get_accuracy(predictions, Y_shuffled)
            print("Accuracy: ", accuracy)

    return W1, b1, W2, b2

In [8]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 30000, 0.025)

Iteration:  0
Accuracy:  0.4991717283268912
Iteration:  1000
Accuracy:  0.5400331308669244
Iteration:  2000
Accuracy:  0.5610160132523467
Iteration:  3000
Accuracy:  0.5759249033683048
Iteration:  4000
Accuracy:  0.5963556046383214
Iteration:  5000
Accuracy:  0.6145775814467145
Iteration:  6000
Accuracy:  0.6322473771397018
Iteration:  7000
Accuracy:  0.6488128106018775
Iteration:  8000
Accuracy:  0.6664826062948647
Iteration:  9000
Accuracy:  0.6858089453340696
Iteration:  10000
Accuracy:  0.6929872998343457
Iteration:  11000
Accuracy:  0.7078961899503037
Iteration:  12000
Accuracy:  0.72170071783545
Iteration:  13000
Accuracy:  0.7310877967973495
Iteration:  14000
Accuracy:  0.7410270568746549
Iteration:  15000
Accuracy:  0.7548315847598012
Iteration:  16000
Accuracy:  0.7614577581446714
Iteration:  17000
Accuracy:  0.7702926559911651
Iteration:  18000
Accuracy:  0.7758144671452236
Iteration:  19000
Accuracy:  0.7774710104914412
Iteration:  20000
Accuracy:  0.781888459414688
Iteratio

In [15]:
#creates another testing set

DATADIR = "/kaggle/input/trainingdata/TrainingData"
CATEGORIES = ["Happy", "Sad"]

# -- switch between testing sets

#DATADIR = "/kaggle/input"
#CATEGORIES = ["testing", "testing2"]

data = []
IMG_SIZE = 50

def create_training_data():
    
    class_num = 0
    
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)  # path to our files
        for img in os.listdir(path):
            img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
            new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
            flattened_pixels = new_array.flatten().tolist()  # Flatten and convert to a Python list

            data.append([class_num] + flattened_pixels)
        class_num += 1
            
create_training_data()

data = np.array(data)
m, n = data.shape
np.random.shuffle(data) # shuffle before splitting into dev and training sets

data_t = data[0:180].T
Y_t = data_t[0]
X_t = data_t[1:n]
X_t = X_t / 255.

In [1]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

def test_prediction(index, W1, b1, W2, b2):
    current_image = X_t[:, index, None]
    prediction = make_predictions(X_t[:, index, None], W1, b1, W2, b2)
    label = Y_t[index]
    print("Prediction: ", prediction)
    print("Label: ", label)
    
    current_image = current_image.reshape((50, 50)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()
    
for i in range(3):
    test_prediction(i, W1, b1, W2, b2)


dev_predictions = make_predictions(X_t, W1, b1, W2, b2)
get_accuracy(dev_predictions, Y_t)

NameError: name 'W1' is not defined

In [17]:
# saves numpy arrays Weights and Biases for use in forward_prop() to make predictions
np.save('Weights1.npy', W1)
np.save('Biases1.npy', b1)
np.save('Weights2.npy', W2)
np.save('Biases2.npy', b2)